# Project Title

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

Questions to consider:

- Who are your stakeholders?
- What are your stakeholders' pain points related to this project?
- Why are your predictions important from a business perspective?
- What exactly is your deliverable: your analysis, or the model itself?
- Does your business understanding/stakeholder require a specific type of model?
    - For example: a highly regulated industry would require a very transparent/simple/interpretable model, whereas a situation where the model itself is your deliverable would likely benefit from a more complex and thus stronger model
   

Additional questions to consider for classification:

- What does a false positive look like in this context?
- What does a false negative look like in this context?
- Which is worse for your stakeholder?
- What metric are you focusing on optimizing, given the answers to the above questions?

## Data Understanding

Describe the data being used for this project.

Questions to consider:

- Where did the data come from, and how do they relate to the data analysis questions?
- What do the data represent? Who is in the sample and what variables are included?
- What is the target variable?
- What are the properties of the variables you intend to use?

In [74]:
# code here to explore your data
import pandas as pd
df_l = pd.read_csv('../../data/Training Set Labels.csv')
df_v = pd.read_csv('../../data/Training Set Values.csv')

## Data Preparation

Describe and justify the process for preparing the data for analysis.

Questions to consider:

- Were there variables you dropped or created?
- How did you address missing values or outliers?
- Why are these choices appropriate given the data and the business problem?
- Can you pipeline your preparation steps to use them consistently in the modeling process?

In [75]:
df = pd.merge(df_v, df_l, on='id')
del df_l

In [76]:
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


In [77]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [78]:
df.funder.value_counts()

Government Of Tanzania            9084
Danida                            3114
Hesawa                            2202
Rwssp                             1374
World Bank                        1349
                                  ... 
Tanzania Egypt Technical Co Op       1
Bhws                                 1
Uniceg                               1
Tadeo                                1
Sadaqatun Jar                        1
Name: funder, Length: 1897, dtype: int64

In [79]:
def funder_top5(row):  
    '''Making top 5 values and setting the rest to 'other'''

    if row['funder']=='Government Of Tanzania':
        return 'Gov'
    elif row['funder']=='Danida':
        return 'Danida'
    elif row['funder']=='Hesawa':
        return 'Hesawa'
    elif row['funder']=='Rwssp':
        return 'Rwssp'
    elif row['funder']=='World Bank':
        return 'World_bank'    
    else:
        return 'other'
    
df['funder'] = df.apply(lambda row: funder_top5(row), axis=1)

In [80]:
str_to_num = {'functional':2, 'functional needs repair':1,
                   'non functional':0}

df['status_group_new']  = df['status_group'].replace(str_to_num)

In [81]:
piv_table = pd.pivot_table(df,index=['funder','status_group'],
                           values='status_group_new', aggfunc='count')
piv_table

status_group_new
funder     status_group                             
Danida     functional                           1713
           functional needs repair               159
           non functional                       1242
Gov        functional                           3720
           functional needs repair               701
           non functional                       4663
Hesawa     functional                            936
           functional needs repair               232
           non functional                       1034
Rwssp      functional                            805
           functional needs repair               109
           non functional                        460
World_bank functional                            545
           functional needs repair                97
           non functional                        707
other      functional                          24540
           functional needs repair              3019
           non functional                      14718

In [82]:
# code here to prepare your data
total_danida = piv_table.loc[('Danida','functional')] + piv_table.loc[('Danida','functional needs repair')] + piv_table.loc[('Danida','non functional')]
percent_functional_danida = (piv_table.loc[('Danida','functional')] / total_danida) * 100

total_gov = piv_table.loc[('Gov','functional')] + piv_table.loc[('Gov','functional needs repair')] + piv_table.loc[('Danida','non functional')]
percent_functional_gov = (piv_table.loc[('Gov','functional')] / total_gov) * 100

total_hesawa = piv_table.loc[('Hesawa','functional')] + piv_table.loc[('Hesawa','functional needs repair')] + piv_table.loc[('Hesawa','non functional')]
percent_functional_hesawa = (piv_table.loc[('Hesawa','functional')] / total_hesawa) * 100

total_rwssp = piv_table.loc[('Rwssp','functional')] + piv_table.loc[('Rwssp','functional needs repair')] + piv_table.loc[('Rwssp','non functional')]
percent_functional_rwssp = (piv_table.loc[('Rwssp','functional')] / total_rwssp) * 100

total_world_bank = piv_table.loc[('World_bank','functional')] + piv_table.loc[('World_bank','functional needs repair')] + piv_table.loc[('World_bank','non functional')]
percent_functional_world_bank = (piv_table.loc[('World_bank', 'functional')] / total_world_bank) * 100

total_other = piv_table.loc[('other', 'functional')] + piv_table.loc[('other', 'functional needs repair')] + piv_table.loc[('other','non functional')]
percent_functional_other = (piv_table.loc[('other','functional')] / total_other) * 100

print('Percent functional danida: ', round(percent_functional_danida,3))
print('Percent functional gov: ', round(percent_functional_gov,3))
print('Percent functional hesawa: ', round(percent_functional_hesawa,3))
print('Percent functional other: ', round(percent_functional_other,3))
print('Percent functional rwssp: ', round(percent_functional_rwssp,3))
print('Percent functional world bank: ', round(percent_functional_world_bank,3))

Percent functional danida:  status_group_new    55.01
dtype: float64
Percent functional gov:  status_group_new    65.69
dtype: float64
Percent functional hesawa:  status_group_new    42.507
dtype: float64
Percent functional other:  status_group_new    58.046
dtype: float64
Percent functional rwssp:  status_group_new    58.588
dtype: float64
Percent functional world bank:  status_group_new    40.4
dtype: float64


In [83]:
df.installer.value_counts()


DWE                     17402
Government               1825
RWE                      1206
Commu                    1060
DANIDA                   1050
                        ...  
Friend from UN              1
Makundya                    1
British                     1
Overland High School        1
AGRICAN                     1
Name: installer, Length: 2145, dtype: int64

In [84]:
def installer_top5(row):
    '''Keep top 5 values and set the rest to 'other'''
    if row['installer']=='DWE':
        return 'DWE'
    elif row['installer']=='Government':
        return 'Gov'
    elif row['installer']=='RWE':
        return 'RWE'
    elif row['installer']=='Commu':
        return 'Commu'
    elif row['installer']=='DANIDA':
        return 'Danida'
    else:
        return 'other'  

df['installer'] = df.apply(lambda row: installer_top5(row), axis=1)

In [85]:
piv_table2 = pd.pivot_table(df,index=['installer','status_group'],
                           values='status_group_new', aggfunc='count')
piv_table2

status_group_new
installer status_group                             
Commu     functional                            724
          functional needs repair                32
          non functional                        304
DWE       functional                           9433
          functional needs repair              1622
          non functional                       6347
Danida    functional                            542
          functional needs repair                83
          non functional                        425
Gov       functional                            535
          functional needs repair               256
          non functional                       1034
RWE       functional                            304
          functional needs repair               137
          non functional                        765
other     functional                          20721
          functional needs repair              2187
          non functional                      13949

In [86]:
total_commu = piv_table2.loc[('Commu', 'functional')] + piv_table2.loc[('Commu', 'functional needs repair')] + piv_table2.loc[('Commu', 'non functional')]
percent_functional_commu = (piv_table2.loc[('Commu', 'functional')] / total_commu) * 100

total_dwe = piv_table2.loc[('DWE', 'functional')] + piv_table2.loc[('DWE', 'functional needs repair')] + piv_table2.loc[('DWE', 'non functional')]
percent_functional_dwe = (piv_table2.loc[('DWE', 'functional')] / total_dwe) * 100

total_rwe = piv_table2.loc[('RWE', 'functional')] + piv_table2.loc[('RWE', 'functional needs repair')] + piv_table2.loc[('RWE', 'non functional')]
percent_functional_rwe = (piv_table2.loc[('Commu', 'functional')] / total_rwe) * 100

total_other = piv_table2.loc[('other', 'functional')] + piv_table2.loc[('other', 'functional needs repair')] + piv_table2.loc[('other', 'non functional')]
percent_functional_other = (piv_table2.loc[('other', 'functional')] / total_other) * 100

print('Percent functional commu: ', round(percent_functional_commu,3))
print('Percent functional dwe: ', round(percent_functional_dwe,3))
print('Percent functional rwe: ', round(percent_functional_rwe,3))
print('Percent functional other: ', round(percent_functional_other,3))



Percent functional commu:  status_group_new    68.302
dtype: float64
Percent functional dwe:  status_group_new    54.206
dtype: float64
Percent functional rwe:  status_group_new    60.033
dtype: float64
Percent functional other:  status_group_new    56.22
dtype: float64


In [87]:
df.subvillage.value_counts()

Madukani        508
Shuleni         506
Majengo         502
Kati            373
Mtakuja         262
               ... 
Ukondamoyo B      1
Fugwe             1
Nyamajiva         1
Mrere             1
Isanga A          1
Name: subvillage, Length: 19287, dtype: int64

In [88]:
print(len(df.subvillage.value_counts()))

19287


In [89]:
df = df.drop('subvillage', axis=1)

In [90]:
df.public_meeting.value_counts()


True     51011
False     5055
Name: public_meeting, dtype: int64

In [91]:
df.public_meeting = df.public_meeting.fillna('Unknown')

In [92]:
df.scheme_management.value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [93]:
def scheme_top5(row):
    '''Keep top 5 values and set the rest to 'other'. '''
    if row['scheme_management']=='VWC':
        return 'VWC'
    elif row['scheme_management']=='WUG':
        return 'WUG'
    elif row['scheme_management']=='Water authority':
        return 'Water Authority'
    elif row['scheme_management']=='WUA':
        return 'WUA'
    elif row['scheme_management']=='Water Board':
        return 'Water Board'
    else:
        return 'other'

df['scheme_management'] = df.apply(lambda row: scheme_top5(row), axis=1)

In [94]:
piv_table3 = pd.pivot_table(df, index=['scheme_management', 'status_group'],
                           values='status_group_new', aggfunc='count')
piv_table3

status_group_new
scheme_management status_group                             
VWC               functional                          18960
                  functional needs repair              2334
                  non functional                      15499
WUA               functional                           1995
                  functional needs repair               239
                  non functional                        649
WUG               functional                           3006
                  functional needs repair               672
                  non functional                       1528
Water Authority   functional                           1618
                  functional needs repair               448
                  non functional                       1087
Water Board       functional                           2053
                  functional needs repair               111
                  non functional                        584
other             functional                           4627
                  functional needs repair               513
                  non functional                       3477

In [95]:
total_vwc = piv_table3.loc[('VWC', 'functional')] + piv_table3.loc[('VWC','functional needs repair')] + piv_table3.loc[('VWC','non functional')]
percent_functional_vwc = (piv_table3.loc[('VWC', 'functional')] / total_vwc) * 100

total_wua = piv_table3.loc[('WUA', 'functional')] + piv_table3.loc[('WUA','functional needs repair')] + piv_table3.loc[('WUA','non functional')]
percent_functional_wua = (piv_table3.loc[('WUA', 'functional')] / total_wua) * 100

total_wug = piv_table3.loc[('WUG', 'functional')] + piv_table3.loc[('WUG','functional needs repair')] + piv_table3.loc[('WUG','non functional')]
percent_functional_wug = (piv_table3.loc[('WUG', 'functional')] / total_wug) * 100

total_wtr_auth = piv_table3.loc[('Water Authority', 'functional')] + piv_table3.loc[('Water Authority','functional needs repair')] + piv_table3.loc[('Water Authority','non functional')]
percent_functional_wtr_auth = (piv_table3.loc[('Water Authority', 'functional')] / total_wtr_auth) * 100

total_wtr_brd = piv_table3.loc[('Water Board', 'functional')] + piv_table3.loc[('Water Board', 'functional needs repair')] + piv_table3.loc[('Water Board', 'non functional')]
percent_functional_wtr_brd = (piv_table3.loc[('Water Authority', 'functional')] / total_wtr_brd) * 100

total_other = piv_table3.loc[('other', 'functional')] + piv_table3.loc[('other', 'functional needs repair')] + piv_table3.loc[('other', 'non functional')]
percent_functional_other = (piv_table3.loc[('other', 'functional')] / total_other) * 100

print('Percent functional other: ', round(percent_functional_other,3))
print('Percent functional vwc: ', round(percent_functional_vwc,3))
print('Percent functional water authority: ', round(percent_functional_wtr_auth,3))
print('Percent functional water board: ', round(percent_functional_wtr_brd,3))
print('Percent functional wua: ', round(percent_functional_wua,3))
print('Percent functional wug: ', round(percent_functional_wug,3))

Percent functional other:  status_group_new    53.696
dtype: float64
Percent functional vwc:  status_group_new    51.532
dtype: float64
Percent functional water authority:  status_group_new    51.316
dtype: float64
Percent functional water board:  status_group_new    58.879
dtype: float64
Percent functional wua:  status_group_new    69.199
dtype: float64
Percent functional wug:  status_group_new    57.741
dtype: float64


In [96]:
df.scheme_name.value_counts()

K                         682
None                      644
Borehole                  546
Chalinze wate             405
M                         400
                         ... 
BL Sia Eki                  1
BL Kilimasimba              1
Islamic                     1
Michee Borehole Scheme      1
Mvaji Ri                    1
Name: scheme_name, Length: 2696, dtype: int64

In [97]:
len(df.scheme_name.unique())

# Lots of factors and the top 5 or so only represent a fraction of the total values. Probably 
# safe to drop this column.

df = df.drop('scheme_name', axis=1)

In [98]:
df.permit.value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [99]:
df.permit = df.permit.fillna('Unknown')

In [100]:
df.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
w

In [101]:
str_cols = df.select_dtypes(include = ['object'])
str_cols.apply(lambda x: len(x.unique()))

date_recorded              356
funder                       6
installer                    6
wpt_name                 37400
basin                        9
region                      21
lga                        125
ward                      2092
public_meeting               3
recorded_by                  1
scheme_management            6
permit                       3
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
status_group                 3
dtype: int64

In [102]:
df.wpt_name.value_counts()

none               3563
Shuleni            1748
Zahanati            830
Msikitini           535
Kanisani            323
                   ... 
Lukwego-Shuleni       1
Lukas Nko             1
Kwa Mzee Justus       1
Dizoza                1
Gedebu                1
Name: wpt_name, Length: 37400, dtype: int64

In [137]:
# Due to the huge number of factors and the lack of a clear dominating value I'll drop this.
df = df.drop('wpt_name', axis=1)

KeyError: "['wpt_name'] not found in axis"

In [104]:
df.basin.value_counts()

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64

In [105]:
piv_table4 = pd.pivot_table(df, index=['basin', 'status_group'],
                           values=['status_group_new'], aggfunc='count')
piv_table4

status_group_new
basin                   status_group                             
Internal                functional                           4482
                        functional needs repair               557
                        non functional                       2746
Lake Nyasa              functional                           3324
                        functional needs repair               250
                        non functional                       1511
Lake Rukwa              functional                           1000
                        functional needs repair               270
                        non functional                       1184
Lake Tanganyika         functional                           3107
                        functional needs repair               742
                        non functional                       2583
Lake Victoria           functional                           5100
                        functional needs repair               989
                        non functional                       4159
Pangani                 functional                           5372
                        functional needs repair               477
                        non functional                       3091
Rufiji                  functional                           5068
                        functional needs repair               437
                        non functional                       2471
Ruvuma / Southern Coast functional                           1670
                        functional needs repair               326
                        non functional                       2497
Wami / Ruvu             functional                           3136
                        functional needs repair               269
                        non functional                       2582

In [106]:
df.region.value_counts()

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [107]:
# Most basins have have more functional than non-functional pumps. Lake Rukwa
# and Ruvuma don't. All the values are over 2000 so this looks like a good feature to keep.
# Region will be considered next.

In [108]:
df.recorded_by.value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

In [109]:
# All data points have the same value so this offers no information that would help build our model.
df = df.drop('recorded_by', axis=1)

In [110]:
df.extraction_type.value_counts()

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [111]:
df.extraction_type_class.value_counts()

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

In [112]:
df.extraction_type_group.value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [113]:
# extraction_type, extraction_type_group and extraction_type_class appear to contain very similar
# data. I'll drop the first two and keep the last one.
df = df.drop(['extraction_type', 'extraction_type_group'], axis=1)

In [114]:
df.management.value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [115]:
# This appears to be almost identical to 'scheme_management'. I'll drop it.

df = df.drop('management', axis=1)

In [116]:
df.management_group.value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

In [117]:
# Appears to offer no new info and is likely to overlap with 'scheme_management'.

df = df.drop('management_group', axis=1)

In [118]:
df.payment.value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [119]:
df.payment_type.value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

In [120]:
# Payment and payment_type contain identical data. Remove one and keep the other.

df = df.drop('payment', 1)

In [121]:
df.water_quality.value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [122]:
df.quality_group.value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

In [123]:
# Water_quality and quality_group contain identical data. Remove one and keep the other.

df = df.drop('quality_group', 1)

In [124]:
df.quantity.value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [125]:
df.quantity_group.value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

In [126]:
# Quantity and quantity_group contain identical data. Remove one and keep the other.

df = df.drop('quantity_group', 1)




In [127]:
df.source.value_counts()



spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [128]:
df.source_class.value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [129]:
df.source_type.value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [130]:
# Source and source_type contain very similar information. Remove one and keep the other.

df = df.drop('source', 1)

In [131]:
str_cols.apply(lambda x: len(x.unique()))

date_recorded              356
funder                       6
installer                    6
wpt_name                 37400
basin                        9
region                      21
lga                        125
ward                      2092
public_meeting               3
recorded_by                  1
scheme_management            6
permit                       3
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
status_group                 3
dtype: int64

In [132]:
df.construction_year.value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [133]:
def construction_year_range(row):
    if row['construction_year'] >= 1960 and row['construction_year'] < 1970:
        return '60s'
    elif row['construction_year'] >= 1970 and row['construction_year'] < 1980:
        return '70s'
    elif row['construction_year'] >= 1980 and row['construction_year'] < 1990:
        return '80s'
    elif row['construction_year'] >= 1990 and row['construction_year'] < 2000:
        return '90s'
    elif row['construction_year'] >= 2000 and row['construction_year'] < 2010:
        return '00s'
    elif row['construction_year'] >= 2010:
        return '10s'
    else:
        return 'unknown'
    
df['construction_year'] = df.apply(lambda row: construction_year_range(row), axis=1)

In [134]:
df.population.describe()

count    59400.000000
mean       179.909983
std        471.482176
min          0.000000
25%          0.000000
50%         25.000000
75%        215.000000
max      30500.000000
Name: population, dtype: float64

In [135]:
df.amount_tsh.describe()

count     59400.000000
mean        317.650385
std        2997.574558
min           0.000000
25%           0.000000
50%           0.000000
75%          20.000000
max      350000.000000
Name: amount_tsh, dtype: float64

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  region                 59400 non-null  object 
 11  region_code            59400 non-null  int64  
 12  district_code          59400 non-null  int64  
 13  lga                    59400 non-null  object 
 14  ward                   59400 non-null  object 
 15  po

## Modeling

Describe and justify the process for analyzing or modeling the data.

Questions to consider:

- How will you analyze the data to arrive at an initial approach?
- How will you iterate on your initial approach to make it better?
- What model type is most appropriate, given the data and the business problem?

## Evaluation

The evaluation of each model should accompany the creation of each model, and you should be sure to evaluate your models consistently.

Evaluate how well your work solves the stated business problem. 

Questions to consider:

- How do you interpret the results?
- How well does your model fit your data? How much better is this than your baseline model? Is it over or under fit?
- How well does your model/data fit any relevant modeling assumptions?

For the final model, you might also consider:

- How confident are you that your results would generalize beyond the data you have?
- How confident are you that this model would benefit the business if put into use?
- What does this final model tell you about the relationship between your inputs and outputs?

### Baseline Understanding

- What does a baseline, model-less prediction look like?

In [65]:
# code here to arrive at a baseline prediction

### First $&(@# Model

Before going too far down the data preparation rabbit hole, be sure to check your work against a first 'substandard' model! What is the easiest way for you to find out how hard your problem is?

In [66]:
# code here for your first 'substandard' model

In [67]:
# code here to evaluate your first 'substandard' model

### Modeling Iterations

Now you can start to use the results of your first model to iterate - there are many options!

In [68]:
# code here to iteratively improve your models

In [69]:
# code here to evaluate your iterations

### 'Final' Model

In the end, you'll arrive at a 'final' model - aka the one you'll use to make your recommendations/conclusions. This likely blends any group work. It might not be the one with the highest scores, but instead might be considered 'final' or 'best' for other reasons.

In [70]:
# code here to show your final model

In [71]:
# code here to evaluate your final model

## Conclusions

Provide your conclusions about the work you've done, including any limitations or next steps.

Questions to consider:

- What would you recommend the business do as a result of this work?
- How could the stakeholder use your model effectively?
- What are some reasons why your analysis might not fully solve the business problem?
- What else could you do in the future to improve this project (future work)?
